In [3]:
import json
sample_in = """
[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]
"""


def get_input(s):
    return [json.loads(x) for x in s.strip().split('\n')]


get_input(sample_in)


[[[[0, [5, 8]], [[1, 7], [9, 6]]], [[4, [1, 2]], [[1, 4], 2]]],
 [[[5, [2, 8]], 4], [5, [[9, 9], 0]]],
 [6, [[[6, 2], [5, 6]], [[7, 6], [4, 7]]]],
 [[[6, [0, 7]], [0, 9]], [4, [9, [9, 0]]]],
 [[[7, [6, 4]], [3, [1, 3]]], [[[5, 5], 1], 9]],
 [[6, [[7, 3], [3, 2]]], [[[3, 8], [5, 7]], 4]],
 [[[[5, 4], [7, 7]], 8], [[8, 3], 8]],
 [[9, 3], [[9, 9], [6, [4, 9]]]],
 [[2, [[7, 7], 7]], [[5, 8], [[9, 3], [0, 2]]]],
 [[[[5, 2], 5], [8, [3, 7]]], [[5, [7, 5]], [4, 4]]]]

In [17]:
def prob1(s):
    snailfishes = get_input(s)
    data = None
    for snailfish in snailfishes:
        snailfish = reduce(snailfish)
        if data is None:
            data = snailfish
        else:
            data = [data, snailfish]
            data = reduce(data)
    return get_magnitude(data)


def get_magnitude(snailfish):
    a, b = snailfish
    if isinstance(a, list):
        a = get_magnitude(a)
    if isinstance(b, list):
        b = get_magnitude(b)
    return 3 * a + 2 * b


assert get_magnitude([[9, 1], [1, 9]]) == 129
assert get_magnitude([[1, 2], [[3, 4], 5]]) == 143
assert get_magnitude([[[[1, 1], [2, 2]], [3, 3]], [4, 4]]) == 445
assert get_magnitude([[[[0, 7], 4], [[7, 8], [6, 0]]], [8, 1]]) == 1384


def reduce(snailfish):
    finished = False
    while not finished:
        snailfish, modified, left_pending, right_pending = explode(snailfish)
        if modified:
            pass
        else:
            snailfish, modified = split(snailfish)
        if not modified:
            finished = True
    return snailfish


def split(snailfish):
    if isinstance(snailfish, int):
        if snailfish >= 10:
            if snailfish % 2 == 0:
                return [snailfish // 2, snailfish // 2], True
            return [snailfish // 2, (snailfish + 1) // 2], True
        else:
            return snailfish, False
    result, modified = split(snailfish[0])
    if modified:
        return [result, snailfish[1]], modified
    result, modified = split(snailfish[1])
    return [snailfish[0], result], modified


assert split(10) == ([5, 5], True)
assert split(11) == ([5, 6], True)
assert split(9) == (9, False)


def explode(snailfish, level=0):
    N = 4
    if isinstance(snailfish, int):
        return snailfish, False, 0, 0
    if level < N:
        left, modified, left_pending, right_pending = explode(
            snailfish[0], level+1)
        if modified:
            right = add_to_leftmost(snailfish[1], right_pending)
            right_pending = 0
        else:
            right, modified, left_pending, right_pending = explode(
                snailfish[1], level + 1)
            if modified:
                left = add_to_rightmost(snailfish[0], left_pending)
                left_pending = 0
        return [left, right], modified, left_pending, right_pending
    # else (level >= N)
    if isinstance(snailfish[0], int) and isinstance(snailfish[1], int):
        modified = True
        left_pending = snailfish[0]
        right_pending = snailfish[1]
        # We are in a leave [int, int] with level >= N
        return 0, modified, left_pending, right_pending
    if isinstance(snailfish[0], list):
        left, modified, left_pending, right_pending = explode(
            snailfish[0], level + 1)
        right = add_to_leftmost(snailfish[1], right_pending)
        right_pending = 0
    else:
        right, modified, left_pending, right_pending = explode(
            snailfish[1], level + 1)
        left = add_to_rightmost(snailfish[0], left_pending)
        left_pending = 0
    return [left, right], modified, left_pending, right_pending


assert explode([5, 5]) == ([5, 5], False, 0, 0)
assert explode([[[[[9, 8], 1], 2], 3], 4]) == (
    [[[[0, 9], 2], 3], 4], True, 9, 0)


def add_to_leftmost(snailfish, data):
    if isinstance(snailfish, int):
        return snailfish + data
    return [add_to_leftmost(snailfish[0], data), snailfish[1]]


def add_to_rightmost(snailfish, data):
    if isinstance(snailfish, int):
        return snailfish + data
    return [snailfish[0], add_to_rightmost(snailfish[1], data)]


test1 = prob1(sample_in)
assert test1 == 4140, test1


In [18]:
prob1(my_in)


3699

In [20]:
import tqdm


def prob2(s):
    snailfishes = get_input(s)
    result = None
    for i, s1 in tqdm.tqdm(enumerate(snailfishes)):
        r1 = reduce(s1)
        for j, s2 in enumerate(snailfishes):
            if i == j:
                continue
            r2 = reduce(s2)
            aux = get_magnitude(reduce([r1, r2]))

            if result is None or result < aux:
                result = aux
    return result


test2 = prob2(sample_in)
assert test2 == 3993
prob2(my_in)


10it [00:00, 124.06it/s]
100it [00:04, 21.57it/s]


4735

In [16]:
my_in = """
[[4,[3,[2,8]]],[[[8,5],4],[[3,5],[4,9]]]]
[[[8,[6,8]],2],[4,[[1,0],[4,5]]]]
[[4,3],[[[3,3],[1,8]],[[0,8],[5,8]]]]
[[[8,[5,8]],[[8,7],[2,1]]],[3,[[0,3],1]]]
[[9,[[9,2],3]],[3,[[9,7],[2,8]]]]
[[[7,[0,0]],[[4,3],[6,3]]],[6,[4,5]]]
[[[[4,2],[0,0]],[4,2]],[[[8,7],8],[[9,4],5]]]
[[4,[1,[1,1]]],[[[0,6],8],[[7,5],[7,3]]]]
[[[1,[9,9]],[4,3]],[[1,[5,8]],9]]
[[2,[6,[6,7]]],[[6,[1,5]],8]]
[4,[[[9,8],[1,3]],[7,[7,4]]]]
[[[[2,6],8],2],[[[2,2],8],[[3,7],8]]]
[[[[1,5],2],[[8,9],[8,3]]],[[[7,3],[1,1]],[[3,7],7]]]
[[[4,4],[8,9]],[5,3]]
[[[[1,4],[4,5]],9],[[2,8],0]]
[[[[4,4],[7,3]],[[9,7],[3,9]]],[9,[5,[5,1]]]]
[[5,[[5,9],6]],[[[5,9],[6,3]],1]]
[[[[1,0],[1,6]],5],[1,9]]
[[[[6,3],[7,2]],1],[[0,6],6]]
[[[3,4],[4,0]],[[[6,4],[2,0]],[7,[7,2]]]]
[[[[1,3],[9,6]],[[1,1],[9,3]]],[[3,8],[[9,5],2]]]
[[[[9,4],6],5],[[[7,6],[2,3]],[6,6]]]
[[6,[2,[0,7]]],[[3,0],[8,[1,9]]]]
[[[6,0],[1,7]],[[[4,2],[5,7]],[[5,0],9]]]
[[[7,[4,9]],0],[[5,[7,5]],[8,[5,1]]]]
[[[[6,1],[8,0]],[3,[4,5]]],[[[8,3],4],[0,2]]]
[[[[3,8],5],[4,8]],9]
[[0,[7,[9,4]]],[2,2]]
[[[[9,6],[4,0]],[1,1]],[3,[[3,6],8]]]
[[[6,[3,3]],[[7,6],3]],[[[2,8],[0,7]],3]]
[[[[3,2],[2,4]],[[8,7],7]],[[[0,2],[1,3]],[8,3]]]
[[[[4,2],[6,8]],6],[[[2,1],[0,3]],[[6,6],[5,6]]]]
[[[[9,0],[1,7]],[[0,3],1]],[[0,[4,3]],7]]
[[[1,[1,4]],1],[[[8,1],9],[[7,1],[7,2]]]]
[2,[2,[[4,2],2]]]
[6,0]
[[[[9,7],7],[2,3]],[[8,[9,4]],[2,3]]]
[[[[4,5],8],6],[5,9]]
[[[5,[6,7]],[[1,9],[8,6]]],7]
[7,[[0,5],4]]
[[[2,4],[2,[1,9]]],[8,[5,6]]]
[3,[[6,[4,8]],[[3,0],9]]]
[[[[4,4],[0,5]],[7,3]],[1,[4,5]]]
[8,[[2,[1,1]],9]]
[[[[5,6],[5,1]],[[7,6],[8,8]]],[2,[[2,1],[3,1]]]]
[[0,[2,[4,6]]],[[[6,0],[3,9]],[0,[1,6]]]]
[[[6,[9,5]],[0,[9,4]]],0]
[[[[5,6],[7,8]],[7,[8,8]]],[[7,[4,7]],[[3,9],7]]]
[1,0]
[[7,2],[9,[3,0]]]
[[[[4,8],9],[1,[0,4]]],[[[5,2],0],8]]
[[[9,[2,5]],[2,[5,8]]],[1,6]]
[[[[0,5],1],[0,4]],7]
[8,[5,9]]
[[[[8,8],[4,8]],[[7,8],7]],[[0,4],8]]
[[[0,6],[9,6]],2]
[[[[2,5],[0,6]],8],[[8,9],1]]
[[[0,9],[1,[1,2]]],[[[4,1],6],7]]
[[[[5,7],[4,6]],[[6,3],[8,2]]],[[[2,5],[0,9]],[5,1]]]
[[[5,0],[[4,5],[6,2]]],[[[1,7],[3,0]],[[8,2],[6,1]]]]
[[[[8,9],9],[9,0]],[4,[[7,2],9]]]
[[[[2,3],[0,5]],[8,8]],[9,[[9,1],8]]]
[[[5,9],[0,[6,2]]],[[3,2],[[1,2],[9,5]]]]
[[[5,2],[8,[0,0]]],[[6,9],[4,[8,4]]]]
[7,3]
[[[6,4],[[0,4],7]],[[5,[0,3]],[8,7]]]
[[1,2],[[3,[5,9]],0]]
[[[6,9],[3,0]],[[[2,1],4],6]]
[[[8,[7,9]],1],[[[2,2],8],8]]
[[[0,1],[6,[3,3]]],5]
[[[[3,9],0],7],2]
[[[3,0],[1,[7,6]]],0]
[[[[3,5],3],8],[[[1,2],[8,8]],[[1,6],[8,1]]]]
[[[9,7],[[1,3],[6,9]]],6]
[[[[1,8],1],[[6,4],[1,8]]],[[[1,7],[5,9]],[[7,0],0]]]
[[[1,[9,3]],[4,0]],7]
[[5,[9,6]],[[4,[0,8]],2]]
[3,[[1,0],[0,2]]]
[[[3,1],[2,7]],[[4,4],5]]
[[8,6],[[4,[5,9]],[[3,7],9]]]
[[[3,2],2],[[3,9],8]]
[[[[4,5],[4,5]],[[0,2],[7,0]]],[[1,4],2]]
[7,[[8,[3,8]],[[5,6],4]]]
[[[[2,8],[2,2]],[[4,4],0]],[[2,[8,0]],2]]
[4,[[[8,8],0],[[1,8],[4,6]]]]
[[[5,[1,2]],7],9]
[[9,[[0,0],9]],[[5,[1,3]],[4,[4,7]]]]
[[3,4],[8,[2,[9,6]]]]
[[[[3,0],1],[[7,6],[5,2]]],[[[9,9],[9,2]],[0,[2,2]]]]
[[[[0,8],[6,1]],[2,0]],[6,[[8,8],[9,1]]]]
[[[5,[1,9]],6],6]
[[[3,5],[[9,9],[2,2]]],[[1,0],4]]
[[4,0],[2,[[8,8],[5,4]]]]
[[6,1],5]
[[[2,[3,7]],0],[1,5]]
[[[[5,9],[5,3]],[6,2]],[[7,9],4]]
[[[5,[0,9]],[[5,6],3]],[3,[7,8]]]
[8,[[[4,6],5],[0,2]]]
[[[0,[6,2]],[6,[6,6]]],[[3,1],6]]
[1,[[6,4],[0,6]]]
"""
